### This jupyter notebook analyzes the  abcd 2.0 data and profile participants for further investigation

In [41]:
# import related libraries
import numpy as np
import pandas as pd
import sklearn

In [42]:
# to avoid memory issue, delete the second row, which is the questionnaire 
infile = open('pdem02.txt','r').readlines()
with open('abcd_demo.txt','w') as outfile:
    for index,line in enumerate(infile):
        """
        find the index not eaqual to one 
        then write the file 
        """
        if index != 1:
            outfile.write(line)

In [46]:
# read data into pandas
# abcd study data file seperate by "\t"
abcd_demo = pd.read_csv('abcd_demo.txt',sep = '\t')

### to clean the data and proflie participants

In [48]:
abcd_demo['subjectkey'].nunique()

11875

#### age

In [49]:
# convert age to years
abcd_demo['age'] = abcd_demo['interview_age']/12

#### Race
1. white
2. africa american
3. hispanic
4. others

In [68]:
abcd_demo['race'] = 4
abcd_demo.loc[(abcd_demo.demo_race_a_p___10 ==1) & 
              (abcd_demo.demo_ethn_v2 != 1) & 
              (abcd_demo.demo_race_a_p___11 !=1), 'race'] = 1
abcd_demo.loc[(abcd_demo.demo_race_a_p___11 == 1) &
              (abcd_demo.demo_ethn_v2 != 1) &
               (abcd_demo.demo_race_a_p___10 != 1), 'race'] =2
abcd_demo.loc[
   abcd_demo.demo_ethn_v2 ==1, 'race'] =3 

In [73]:
abcd_demo['race'].value_counts()

1    6724
3    2409
2    1860
4     882
Name: race, dtype: int64